In [7]:
import re
import sys
import itertools

def keymap_lookup(keymap, nums):
    ret = []
    for num in nums:
        for k, v in keymap.items():
            if k == num:
                ret.append(v)
    #print(ret)       
    return ret

def read_content(filename='words.txt'):
    with open(filename, 'r') as f:
        return f.read().strip().replace(',', ' ')

#make a dictionary mapping a word to its frequency
def parse_content(content):
    content = content.split()
    word_dict = {}
    i = 0
    #save word-frequency pairs as dict
    while(i < len(content)):
        word_dict[content[i]] = content[i + 1]
        i += 2
    #print(word_dict)
    return word_dict

def ask_for_numbers():
    while True:
        response = input('What numbers have you pressed? ').strip()
        if len(response) < 3:
            print('You need to enter at least three numbers.', file=sys.stderr)
        elif re.search("[^2-9]", response):
            print("You entered a character that isn't one of 2, 3, 4, 5, 6, 7, 8, or 9. Please try again.", file=sys.stderr)
        else:
            return response

#return a dictionary mapping letters/words 
#in a tree structure
def make_tree(words):
    tree = {}
    for word, number in words.items():
        subdict = tree
        for letter in word:
            if letter not in subdict:
                subdict[letter] = {}
            subdict = subdict[letter]
        #end-of-word 
        subdict['$' + word] = number
    #print(tree)
    return tree
        
#return all nodes with words that have given prefix (as dict)
def search(tree, prefix):
    subdict = tree
    for letter in prefix:
        if letter in subdict:
            subdict = subdict[letter]
        else:
            return 0
    return subdict

#unnest dictionary and return
#list of all keys and values
def flatten_dict(d):    
    res = []  
    if isinstance(d, dict):
        for key, val in d.items():
            res.extend(flatten_dict(val))
            res.extend(flatten_dict(key))
    else:
        res.append(d)        
    return res

#look up words for prefixes from T9 numbers entered by user
def predict(tree, numbers, keymap):
    #get possible letters for numbers
    t9_letters = keymap_lookup(keymap, numbers)
    print(t9_letters)
    #get all possible combinations
    combis = list(itertools.product(*t9_letters)) #itertools.product() is used to find the cartesian product from the given iterator
    
    res = []
    freqs = []
    #search the tree for the prefixes
    for c in combis:
        node = search(tree, c)
        if node != 0:
            for item in flatten_dict(node):
                #get whole words out of unnested dict without preceding '$'
                if type(item) == str and item[0] == '$':
                    res.append(item[1:])
                #get frequencies out of unnested dict
                if len(item) > 1 and item[0] != '$':
                    freqs.append(item)
                    
    result = list(zip(res, freqs))
    return sorted(result, key=lambda x: int(x[1]), reverse=True)

In [9]:
keymap = {
    '2': 'abc',
    '3': 'def',
    '4': 'ghi',
    '5': 'jkl',
    '6': 'mno',
    '7': 'pqrs',
    '8': 'tuv',
    '9': 'wxyz',
}

content = read_content(filename='ngrams-10k.txt')

words = parse_content(content)

tree = make_tree(words)

while True:
    numbers = ask_for_numbers()
    predictions = predict(tree, numbers, keymap)

    if not predictions:
        print('No words were found that match those numbers. :(')
    else:
        for prediction in predictions:
            print(prediction)

    answer = input('Do you want to go again? [y/n]\n')
    again = answer and answer[0] in ('y', 'Y')
    if not again:
        break

What numbers have you pressed? 345
['def', 'ghi', 'jkl']
('filled', '8209395')
('fill', '3718553')
('film', '3700041')
('file', '3027919')
('filling', '1784479')
('filter', '1517557')
('films', '1509898')
('filed', '1267837')
('files', '1187347')
('dilute', '878374')
('diligence', '830551')
('fills', '776693')
('dilemma', '741052')
('filing', '642537')
Do you want to go again? [y/n]
n


In [ ]:
words = {
  'ban': 10,
  'band': 5,
  'bar': 14,
  'can': 32,
  'candy': 7
}

{
   "b": {
      "a": {
         "n": {
            "$ban": 10,
            "d": {
               "$band": 5
            }
         },
         "r": {
            "$bar": 14
         }
      }
   },
   "c": {
      "a": {
         "n": {
            "$can": 32,
            "d": {
               "y": {
                  "$candy": 7
               }
            }
         }
      }
   }
}